In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nibabel as nib
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
import datetime
import pandas as pd
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset
import random

In [2]:
# Set seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
import importlib.util
import sys

# Specify the full path to the module file
module_path = 'D:\\Github Folder\\MasterThesis\\Code\\FUNCTIONS.py'

# Load the module
spec = importlib.util.spec_from_file_location("FUNCTIONS", module_path)
functions = importlib.util.module_from_spec(spec)
spec.loader.exec_module(functions)

# Now you can use the functions as if you had imported them
load_datasets = functions.load_datasets
create_dataloaders = functions.create_dataloaders
train_and_validate = functions.train_and_validate
test_model = functions.test_model


In [4]:
df = pd.read_excel(r"references\NEW_COMBINED_FINAL_Subject_info.xlsx")

In [5]:
# Assuming 'df' is your DataFrame loaded with the 'Research Group' column available
label_categories = pd.Categorical(df['Research Group'])
label_mapping = {code: category for code, category in enumerate(label_categories.categories)}

In [28]:
import torch
import torch.nn as nn

class VGG3D(nn.Module):
    def __init__(self, num_classes=1, input_shape=(1, 193, 229, 193)):
        super(VGG3D, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channels=input_shape[0], out_channels=8, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=8, out_channels=8, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=3)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(in_channels=8, out_channels=16, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=16, out_channels=16, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=3)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv3d(in_channels=16, out_channels=32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        )

        self.conv4 = nn.Sequential(
            nn.Conv3d(in_channels=32, out_channels=64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2,2,2), stride=2)
        )
        
        # This will be updated after running the model to find the correct flattened size
        self.fc1 = nn.Sequential(
            nn.Linear(9600, 128),  # This number will be updated
            nn.BatchNorm1d(128),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        if(num_classes==2):
            self.out = nn.Linear(64, 2)
            self.out_act = nn.Sigmoid()
        else:
            self.out = nn.Linear(64, num_classes)
            self.out_act = nn.Softmax(dim=1)

    def forward(self, x, drop_prob=0.8):
       # print("Input shape:", x.shape)
        x = self.conv1(x)
        #print("After Conv1:", x.shape)
        x = self.conv2(x)
       # print("After Conv2:", x.shape)
        x = self.conv3(x)
       # print("After Conv3:", x.shape)
        x = self.conv4(x)
       # print("After Conv4:", x.shape)
        x = x.view(x.size(0), -1)
       #print("Shape before FC1:", x.shape)
        x = self.fc1(x)
        x = nn.Dropout(drop_prob)(x)
        x = self.fc2(x)
        prob = self.out(x)
        return prob

In [29]:
def select_optimizer(model, config):
    """Select optimizer based on configuration."""
    lr = config.get('lr', 0.001)  # Default learning rate
    weight_decay = config.get('weight_decay', 0)  # Default weight decay
    
    if config['optimizer'] == 'Adam':
        return optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif config['optimizer'] == 'SGD':
        momentum = config.get('momentum', 0.9)  # Default momentum for SGD
        return optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise ValueError("Unsupported optimizer")

def select_criterion(config):
    """Select loss criterion based on configuration."""
    if config['loss_criterion'] == 'Cross-Entropy':
        return nn.CrossEntropyLoss()
    elif config['loss_criterion'] == 'BCEWithLogits':
        return nn.BCEWithLogitsLoss()
    else:
        raise ValueError("Unsupported loss criterion")
    
    

In [32]:
from openpyxl import load_workbook
import pandas as pd
import os
import datetime
import torch
import torch.nn as nn
import torch.optim as optim






def run_experiment(df, config):
    train_dataset, val_dataset, test_dataset = load_datasets(df, image_type = config['image_type'], sample_size = config['sample_size'], loss_type = config['loss_criterion'])
    train_loader, val_loader, test_loader = create_dataloaders(train_dataset, val_dataset, test_dataset, batch_size=config['batch_size'])
    
    device = torch.device("cuda")
    model = VGG3D().to(device)
    
    criterion = select_criterion(config)
    optimizer = select_optimizer(model, config)


    # Training and validation
    train_accuracies, val_accuracies, val_losses, results_df = train_and_validate(model, train_loader, val_loader, criterion, optimizer, config['num_epochs'], config['patience'], device, config['loss_criterion'])
    test_results, test_accuracy = test_model(model, test_loader, label_mapping, device, loss_type = config['loss_criterion'])
    
    # Save detailed results to Excel
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime('%Y-%m-%d_%H-%M-%S')
    report_filename = os.path.join('reports', f'{formatted_time}_Experiment.xlsx')
    onnx_filename = os.path.join('models', f'{formatted_time}_Model.')
          # Save the model to ONNX


    
    summary_data = {
        'Phase': ['Training', 'Validation', 'Testing'],
        'Accuracy': [train_accuracies[-1], val_accuracies[-1], test_accuracy]
    }
    summary_df = pd.DataFrame(summary_data)
    all_results = pd.DataFrame(test_results)
    config_df = pd.DataFrame([config])
    
    with pd.ExcelWriter(report_filename) as writer:
        config_df.to_excel(writer, sheet_name='Configuration')
        all_results.to_excel(writer, sheet_name='Results')
        summary_df.to_excel(writer, sheet_name='Summary')
        results_df.to_excel(writer, sheet_name='Training_Results')

# Append a summary of this experiment to the cumulative RESULTS.xlsx file
    results_file = os.path.join('reports', 'RESULTS.xlsx')
    experiment_summary = {**config, **{'Training Accuracy': train_accuracies[-1], 'Validation Accuracy': val_accuracies[-1], 'Test Accuracy': test_accuracy, 'DATETIME': formatted_time}}
    summary_row = pd.DataFrame([experiment_summary])

    if os.path.exists(results_file):
        with pd.ExcelWriter(results_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            existing_df = pd.read_excel(results_file)
            combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
            combined_df = combined_df.reindex(columns=(existing_df.columns.tolist() + [col for col in summary_row.columns if col not in existing_df.columns]))
            combined_df.to_excel(writer, index=False, sheet_name='Sheet1')
    else:
        summary_row.to_excel(results_file, index=False)

    dummy_input = torch.randn(4, 1, 193, 229, 193, device=device)  # Adjust size according to your model's input
    torch.onnx.export(model, dummy_input, onnx_filename, export_params=True)

    return report_filename, train_accuracies[-1], val_accuracies[-1], test_accuracy


# Example configuration and use case
config = {
    'optimizer': 'Adam',
    'lr' : 0.01,
    'loss_criterion': 'BCEWithLogits',
    'num_epochs': 100,
    'batch_size': 4,
    'patience': 20,
    'Description' : 'REAL DIMENSIONS',
    'image_type' : 'Resampled Images_fused',
    'weight_decay' : 0,
    'momentum' : 0.7,
    'sample_size' : None

}

In [33]:
  #column_mapping = {
  #          'Co-registered PET': 'Co-registered PET',
   #         'Fused Images': 'Fused Images',
    #        'Masked PET': 'Masked PET',
     #       'Spatial Normalization': 'Spatial Normalization',
      #      'Resampled Images(Co-registered PET)': 'Resampled Images(Co-registered PET)',
       #     'Resampled Images(Masked PET)': 'Resampled Images(Masked PET)',
        #    'Resampled Images(Spatial Normalization)': 'Resampled Images(Spatial Normalization)',
         #   'Resampled Images_fused': 'Resampled Images_fused'
        #}

image_types = ['Fused Images', 'Co-registered PET', 'Masked PET', 'Spatial Normalization']
results = []
for image_type in image_types:
    config['image_type'] = image_type
    print(f"Working on image type: {image_type}")
    result = run_experiment(df, config)
    results.append(result)



Working on image type: Fused Images


Epoch 1/100 - Train: 100%|██████████| 78/78 [19:17<00:00, 14.85s/it]


Epoch 1: Train Loss: 0.7337 - Train Accuracy: 49.68%


Epoch 1/100 - Validate: 100%|██████████| 11/11 [01:50<00:00, 10.02s/it]


Epoch 1: Validation Loss: 0.7122 - Validation Accuracy: 59.09%


...

Epoch 23: Validation Loss: 0.6925 - Validation Accuracy: 47.73%
Early stopping triggered after 23 epochs due to no improvement in validation loss or accuracy.


Testing: 100%|██████████| 22/22 [03:17<00:00,  9.00s/it]
C:\Users\Micha\AppData\Local\Temp\ipykernel_46428\2820527412.py:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([existing_df, summary_row], ignore_index=True)
c:\Users\Micha\.pyenv\pyenv-win\versions\3.10.10\lib\site-packages\torch\onnx\symbolic_helper.py:1515: UserWarning: ONNX export mode is set to TrainingMode.EVAL, but operator 'dropout' is set to train=True. Exporting with train=True.
  warnings.warn(


Working on image type: Co-registered PET


Epoch 1/100 - Train:  35%|███▍      | 27/78 [06:39<12:35, 14.80s/it]


KeyboardInterrupt: 